In [37]:
# Linear Regression

In [38]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.metrics import mean_squared_error, r2_score

set_config(display="diagram")

df = pd.read_csv(r'C:\Users\viksaraw\OneDrive\Berkeley\Capstone-1\May\Data\AfterEDA.csv')
df.head(5)
#df.shape

,date_sold,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [39]:
# Data cleanup, convert date_sold to just year
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date_sold      21613 non-null  object 
 1   price          21613 non-null  float64
 2   bedrooms       21613 non-null  int64  
 3   bathrooms      21613 non-null  float64
 4   sqft_living    21613 non-null  int64  
 5   sqft_lot       21613 non-null  int64  
 6   floors         21613 non-null  float64
 7   waterfront     21613 non-null  int64  
 8   view           21613 non-null  int64  
 9   condition      21613 non-null  int64  
 10  grade          21613 non-null  int64  
 11  sqft_above     21613 non-null  int64  
 12  sqft_basement  21613 non-null  int64  
 13  yr_built       21613 non-null  int64  
 14  yr_renovated   21613 non-null  int64  
 15  zipcode        21613 non-null  int64  
 16  lat            21613 non-null  float64
 17  long           21613 non-null  float64
 18  sqft_l

In [40]:
# Convert date sold to just year_sold
df['year_sold'] = pd.to_datetime(df['date_sold']).dt.year.astype(int)
df = df.drop(columns=['date_sold'], axis =1)


In [41]:
# Define features and target
X = df.drop(columns=['price'], axis =1)
y = df['price']

# Train Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

# Define the pipeline with StandardScaler and Linear Regression
pipeline_linear = Pipeline([
    ('scaler', StandardScaler()),  # Normalize the data
    ('lr', LinearRegression())  # Apply Linear Regression
])

# Define hyperparameter grid for LinearRegression (only 'fit_intercept' and 'normalize' are available)
param_grid = {
    'lr__fit_intercept': [True, False]
}

# Initialize variables to store the best CV value and corresponding MSE
best_cv = None
best_mse = float('inf')

# Loop through CV values from 2 to 11
for cv in range(2, 11):
    grid_search = GridSearchCV(pipeline_linear, param_grid, cv=cv, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)

 # Best parameters and model evaluation
    best_model = grid_search.best_estimator_

    
    # Make predictions
    # y_pred = best_model.predict(X_test)
    #Y_pred_test = best_mode.predict(X_train)

    # Predictions
    y_pred_train = best_model.predict(X_train)
    y_pred_test = best_model.predict(X_test)
    


In [43]:
    # Metrics
    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)
    rmse_train = mse_train ** 0.5
    rmse_test = mse_test ** 0.5
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)
    print(f"CV={cv}, Best Params: {grid_search.best_params_}")
    print(f"Train R²: {r2_train:.3f}, Test R²: {r2_test:.3f}")
    print(f"Train RMSE: {rmse_train:.3f}, Test RMSE: {rmse_test:.3f}")


print("-" * 60)
    if mse_test < best_mse:
        best_cv = cv
        best_mse = mse_test
print(f"Best CV: {best_cv}, Best Test MSE: {best_mse:.3f}")

CV=10, Best Params: {'lr__fit_intercept': True}
Train R²: 0.701, Test R²: 0.703
Train RMSE: 197757.408, Test RMSE: 212040.834
------------------------------------------------------------
Best CV: 10, Best Test MSE: 44961315278.455
